<a href="https://colab.research.google.com/github/perlacueto/BecomingHuman/blob/master/Clasificaci%C3%B3n_Red_Neuronal_Sentimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carga Bibliotecas

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 15.2 MB/s 


In [4]:
import pandas as pd

# Matplot
import matplotlib.pyplot as plt
%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
#xgboost
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import load_model

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
#from nltk.stem import WordNetLemmatizer

# Word2vec
import gensim
from gensim.models import Word2Vec


# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
DATASET_ENCODING = "ISO-8859-1"

SEQUENCE_LENGTH = 600

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

# EXPORT
KERAS_MODEL = "/content/drive/MyDrive/Mercantil/model.h5"
WORD2VEC_MODEL = "/content/drive/MyDrive/Mercantil/model.w2v"
TOKENIZER_MODEL = "/content/drive/MyDrive/Mercantil/tokenizer.pkl"
ENCODER_MODEL = "/content/drive/MyDrive/Mercantil/encoder.pkl"

## Carga modelo de palabras

In [6]:

w2v_model = Word2Vec.load(WORD2VEC_MODEL)

2022-06-08 14:40:07,249 : INFO : loading Word2Vec object from /content/drive/MyDrive/Mercantil/model.w2v
2022-06-08 14:40:07,720 : INFO : loading wv recursively from /content/drive/MyDrive/Mercantil/model.w2v.wv.* with mmap=None
2022-06-08 14:40:07,724 : INFO : setting ignored attribute vectors_norm to None
2022-06-08 14:40:07,735 : INFO : loading vocabulary recursively from /content/drive/MyDrive/Mercantil/model.w2v.vocabulary.* with mmap=None
2022-06-08 14:40:07,741 : INFO : loading trainables recursively from /content/drive/MyDrive/Mercantil/model.w2v.trainables.* with mmap=None
2022-06-08 14:40:07,743 : INFO : setting ignored attribute cum_table to None
2022-06-08 14:40:07,746 : INFO : loaded /content/drive/MyDrive/Mercantil/model.w2v


Pregunta vocabulario

In [7]:
w2v_model.most_similar("preguntar")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2022-06-08 14:40:07,771 : INFO : precomputing L2-norms of word weight vectors


[('deber', 0.30300432443618774),
 ('reportar', 0.22915126383304596),
 ('completar', 0.21649152040481567),
 ('decir', 0.2083631455898285),
 ('justicia', 0.2032536268234253),
 ('robar', 0.20222626626491547),
 ('buscar', 0.20163574814796448),
 ('eficaz', 0.1849234700202942),
 ('no', 0.1840522587299347),
 ('ingresar', 0.17772991955280304)]

# Modelo análisis de sentimiento

### Cargar modelo análisis de Sentimiento

In [20]:
with open(TOKENIZER_MODEL, 'rb') as handle:
    tokenizer = pickle.load(handle)

In [9]:
file = open(ENCODER_MODEL,'rb')
file.seek(0)
tokenizer = pickle.load(file)
file.close()

In [21]:
tokenizer

Debe aparecer: <keras_preprocessing.text.Tokenizer at ...>


Si no vuelve a ejecutar la celda 

In [11]:
model = load_model(KERAS_MODEL)

In [12]:
def decode_sentiment(score, include_neutral=False):
    if include_neutral:        
        label = NEUTRAL
        if score <= SENTIMENT_THRESHOLDS[0]:
            label = NEGATIVE
        elif score >= SENTIMENT_THRESHOLDS[1]:
            label = POSITIVE

        return label
    else:
        return NEGATIVE if score < 0.5 else POSITIVE

In [13]:
def predict(text, include_neutral=False):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=SEQUENCE_LENGTH)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}  

## Clasificación Comentarios

### **Cargar Herramientas**

In [14]:
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
#from nltk.stem import WordNetLemmatizer

#Replace all non alphabets
from unidecode import unidecode

#For lematization
import spacy


In [15]:
nltk.download('wordnet')
nltk.download('stopwords')
stop_words = stopwords.words("spanish")
stemmer = SnowballStemmer("spanish")#extrae la raíz de una palabra
!spacy download es
lematiza = spacy.load("es")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.2 MB 1.6 MB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.2.5-py3-none-any.whl size=16172933 sha256=a873134961f45e17add010aa3ea6a7e76717ccc97fd231e0305bc5c510d934ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-tas62sjl/wheels/21/8d/a9/6c1a2809c55dd22cd9644ae503a52ba6206b04aa57ba83a3d8
Successfully built es-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')


In [16]:
palabras = {'madurar':'maduro','clavar':'clave','paginar':'pagina','sera':'ser','seguira':'seguir','divisar':'divisa',
            'lonuevo':'nuevo','deberiar':'deber','cumpliraa+-os':'cumpleanos','nadar':'nada','pudrir':'poder',
            'sii':'si','ye':'si','parir':'para','cariaa+-o':'carino','app':'aplicacion','gracia':'gracias',
            'fallir':'fallar','aa+-os':'ano','huma?anaird':'humanidad','linear':'linea','contactarer':'contactar',
            'naciinal':'nacional','ma?anada':'mañana','estarfaeued':'estar','eaa':'esa','quebhay':'que hay','enviir':'enviar',
            'mercanril':'mercantil','estrellar':'estrella','malisimir':'malo','gustariar':'gustar','cencilla':'sencillo',
            'sencillar':'sencillo'
           }

In [17]:
emojis = {u'\U0001F3E6':' banco',u'\U0001F601':' sonrisa', u'\U0001F602':' risa', u'\U0001F603':' sonrisa', u'\U0001F604':' sonrisa',u'\U0001F605':' sonrisa',
          u'\U0001F609':' guiño',u'\U0001F60A':' sonrisa', u'\U0001F612':' desagrado',u'\U0001F613':' desagrado', u'\U0001F614':' triste',u'\U0001F616':' triste',
          u'\U0001F618':' beso',u'\U0001F61A':' beso', u'\U0001F61C':' burla', u'\U0001F61D':' burla', u'\U0001F61E':' triste', u'\U0001F620':' enojo',
          u'\U0001F621':' enojo', u'\U0001F622':' triste', u'\U0001F623':' frustración', u'\U0001F624':' enojo', u'\U0001F625':' preocupado', 
          u'\U0001F629':' frustrado', u'\U0001F62A':' triste', u'\U0001F62D':' triste', u'\U0001F62B':' cansado',u'\U0001F630':' preocupado',
          u'\U0001F633':' sonrojado',u'\U0001F638':' risa',u'\U0001F639':' risa', u'\U0001F63A':' risa', u'\U0001F63B':' me encanta', u'\U0001F60B':' me encanta',
          u'\U0001F63F':' triste', u'\U0001F645':' bloqueo',  u'\U0001F645':' bloqueo', u'\U0001F64F':' rezar', u'\U00002764':' corazón', u'\U0001F44D':' bien',
          u'\U0001F44E':' mal',  u'\U0001F44F':' aplauso', u'\U0001F46A':' familia', u'\U0001F47B':' fantasma', u'\U0001F47F':' enojado', u'\U0001F4A9':' mierda',
          u'\U0001F4AA':' fuerza', u'\U0001F610':' neutral', u'\U0001F611':' serio', u'\U0001F615':' confundido', u'\U0001F61F':' preocupado', 
          u'\U0001F627':' angustiado', u'\U0001F31E':' sol', u'\U0001f1fb\U0001f1ea':' Venezuela', u'\U0001F628':' miedo', u'\U0001F631':' asustado',
          u'	\U0001f926\u200D\u2642\uFE0F':' hombremanocara', u':)': 'sonrisa', ':-)': 'sonrisa', ';d': 'guiño', u'\U0001F926':'mujermanocara',
          ':-E': 'vampiro', ':(': 'triste', '@MercantilBanco' : 'MercantilBanco',
          ':-(': 'triste', ':-<': 'triste', ':P': 'burla', ':O': 'sorpresa',
          ':-@': 'shock', ':@': 'shock',':-$': 'confusion', ':\\': 'molesto', 
          ':#': 'mudo', ':X': 'mudo', ':^)': 'sonrisa', ':-&': 'confundido', '$_$': 'avaro',
          '@@': 'indiferente', ':-!': 'confundido', ':-D': 'sonrisa', ':-0': 'gritar', 'O.o': 'confundido',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'triste', ';)': 'guiño', 
          ';-)': 'guiño', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'fantasma', '=^.^=': 'gato'}

In [18]:
def preprocess(tweet, stem=False):
    
    # Create Lemmatizer and Stemmer.
    #wordLemm = WordNetLemmatizer()
    
    # Defining regex patterns.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    tweet = tweet.lower()
        
    # Replace all URls with 'URL'
    tweet = re.sub(urlPattern,' URL',tweet)
    # Replace all emojis.
    for emoji in emojis.keys():
      tweet = tweet.replace(emoji, "EMOJI" + emojis[emoji]+" ")        
    # Replace @USERNAME to 'USER'.
    tweet = re.sub(userPattern,'', tweet)        
    # Replace all non alphabets.
    tweet = unidecode(tweet)
    # Replace 3 or more consecutive letters by 2 letter.
    tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
    #Lematiza
    tweet = lematiza(tweet)
    lemmas = [tok.lemma_.lower() for tok in tweet]
    tokens = []
    for token in lemmas:
          # Checking if the word is a stopword.
          if token not in stop_words:
            #cheching if word has more than one letter
            for palabra in palabras.keys():
              token = token.replace(palabra, palabras[palabra])
            if len(token)>1:
              if stem:
                tokens.append(stemmer.stem(token))
              else:
                tokens.append(token)
    return " ".join(tokens)


        

### **Predicción Simple**

In [115]:
predict(preprocess("Venezuela es el único país donde solicitar la reposición de una tarjeta te dicen que es un favor y no un servicio. Gracias @MercantilBanco"))

{'elapsed_time': 0.12153840065002441,
 'label': 'NEGATIVE',
 'score': 0.4928320050239563}

In [38]:
predict(preprocess('Gracias'))

{'elapsed_time': 0.11188554763793945,
 'label': 'POSITIVE',
 'score': 0.6223584413528442}

In [25]:
predict(preprocess("deberia ir a una oficina para cambiar mi numero de telefono?"))

{'elapsed_time': 0.13450002670288086,
 'label': 'POSITIVE',
 'score': 0.5431056618690491}

In [118]:
predict(preprocess("Estoy encantada con la atención que recibí"))

{'elapsed_time': 0.10958695411682129,
 'label': 'POSITIVE',
 'score': 0.5502634644508362}

In [123]:
predict(preprocess("Ayer me depositaron, desde tpago"))

{'elapsed_time': 0.11563849449157715,
 'label': 'POSITIVE',
 'score': 0.6358295679092407}

### **Inspecciona tabla de comentarios**

 Cargar tabla para analizar

In [72]:
comentarios = pd.read_excel('/content/drive/MyDrive/Mercantil/analisis_influencer/comen_top_feb.xlsx')

In [75]:
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      57 non-null     float64       
 1   screen_name  57 non-null     object        
 2   created_at   57 non-null     datetime64[ns]
 3   text         57 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.9+ KB


**Debe tener una columna con nombre 'text'...**
['user_id', 'screen_name', 'created_at', 'text']

In [44]:
comentarios.columns

Index(['user_id', 'screen_name', 'created_at', 'text'], dtype='object')

**Limpia data**

In [74]:
comentarios = comentarios.drop(['Unnamed: 0'], axis=1) #elimina alguna columna en especial

In [ ]:
df = df.dropna() #elimina nulos

In [ ]:
comentarios = comentarios.drop_duplicates()#elimina duplicados

**Agregar columnas que tendrán la clasificación del sentimiento:** Target y Score

In [76]:
comentarios = comentarios.assign(text_process=np.nan) #texto lematizado
comentarios = comentarios.assign(target=np.nan) #clasificación
comentarios = comentarios.assign(score=np.nan) #score
comentarios.head()

,user_id,screen_name,created_at,text,text_process,target,score
0,1.913194e+08,TitoCorona9,2022-02-24 15:55:14,Cual consideran el mejor banco de Venezuela y ...,NaN,NaN,NaN
1,4.652935e+09,ElGaboRD,2022-02-03 21:53:15,Listado de Quiebra de Bancos de los gobiernos ...,NaN,NaN,NaN
2,1.414676e+18,RealSeasonNT1,2022-02-10 20:02:40,Esta parte de la cola de emisión de bonos del ...,NaN,NaN,NaN
3,1.926909e+08,granfran5012,2021-10-17 00:24:22,"Si sospechas que has sido víctima de fraude, d...",NaN,NaN,NaN
4,1.926909e+08,granfran5012,2022-02-23 17:48:41,@granfran5012 @MercantilBanco @SudebanInforma ...,NaN,NaN,NaN


### **Procesar texto en tabla**

In [99]:
comentarios.text_process = comentarios.text.apply(lambda x: str(x)) #convierte en texto
comentarios.text_process = comentarios.text.apply(lambda x: preprocess(x)) # lematiza
comentarios.head() #muestra

,user_id,screen_name,created_at,text,text_process,target,score
0,1.913194e+08,TitoCorona9,2022-02-24 15:55:14,Cual consideran el mejor banco de Venezuela y ...,considerar mejor banco venezuela mercantil,NaN,NaN
1,4.652935e+09,ElGaboRD,2022-02-03 21:53:15,Listado de Quiebra de Bancos de los gobiernos ...,listar quebrar banco gobierno prd prm ano medi...,NaN,NaN
2,1.414676e+18,RealSeasonNT1,2022-02-10 20:02:40,Esta parte de la cola de emisión de bonos del ...,partir cola emision bono banco mercantil cibel...,NaN,NaN
3,1.926909e+08,granfran5012,2021-10-17 00:24:22,"Si sospechas que has sido víctima de fraude, d...",si sospechar haber ser victimar fraude deber r...,NaN,NaN
4,1.926909e+08,granfran5012,2022-02-23 17:48:41,@granfran5012 @MercantilBanco @SudebanInforma ...,ir banesco abri alli abrir contar ahorrar s...,NaN,NaN


**Clasificar comentarios de DataFrame**

In [102]:
comentarios.score = comentarios.text_process.apply(lambda x: predict(x)['score']) 
comentarios.target = comentarios.text_process.apply(lambda x: predict(x)['label'])
comentarios.head()

,user_id,screen_name,created_at,text,text_process,target,score
0,1.913194e+08,TitoCorona9,2022-02-24 15:55:14,Cual consideran el mejor banco de Venezuela y ...,considerar mejor banco venezuela mercantil,POSITIVE,0.532526
1,4.652935e+09,ElGaboRD,2022-02-03 21:53:15,Listado de Quiebra de Bancos de los gobiernos ...,listar quebrar banco gobierno prd prm ano medi...,POSITIVE,0.529752
2,1.414676e+18,RealSeasonNT1,2022-02-10 20:02:40,Esta parte de la cola de emisión de bonos del ...,partir cola emision bono banco mercantil cibel...,POSITIVE,0.581867
3,1.926909e+08,granfran5012,2021-10-17 00:24:22,"Si sospechas que has sido víctima de fraude, d...",si sospechar haber ser victimar fraude deber r...,POSITIVE,0.551686
4,1.926909e+08,granfran5012,2022-02-23 17:48:41,@granfran5012 @MercantilBanco @SudebanInforma ...,ir banesco abri alli abrir contar ahorrar s...,POSITIVE,0.577103


### **REVISIÓN**

In [103]:
comentarios.groupby('target').count() # cuenta según clasificación

,user_id,screen_name,created_at,text,text_process,score
target,,,,,,
NEGATIVE,31,31,31,31,31,31
POSITIVE,26,26,26,26,26,26


In [ ]:
comentarios.loc[comentarios['target']== 'NEGATIVE'] #muestra comentarios negativos

In [112]:
print(comentarios['target'][21]) #muetra clasificación particular
comentarios['text'][21] #muestra comentarios particular

'Venezuela es el único país donde solicitar la reposición de una tarjeta te dicen que es un favor y no un servicio. Gracias @MercantilBanco'

## Guarda excel nuevo

In [ ]:
comentarios.to_excel('/content/drive/MyDrive/Mercantil/analisis_influencer/comen_class_feb.xlsx') #Escribir nombre y ruta de preferencia